In [43]:
import pandas as pd
import plotly.graph_objects as go
import os

In [44]:
raw_data = pd.read_csv('data.csv')
raw_data

,Name,Week,Score Projected,Score Actual,Bench Projected,Bench Actual,QB Projected,QB Actual,RB1 Projected,RB1 Actual,...,WR2 Actual,TE Projected,TE Actual,FLEX Projected,FLEX Actual,D Projected,D Actual,K Projected,K Actual,Win
0,Didi,1,126.6,108.48,100.5,100.30,23.4,24.28,17.7,16.5,...,4.7,9.9,14.9,14.2,13.9,6.2,2,8.2,13,0
1,Luk,1,132.6,131.56,86.8,80.72,22.0,29.36,17.3,14.5,...,10.1,13.6,12.5,14.9,11.6,7.8,16,8.2,1,1
2,Derek,1,126.3,105.32,62.5,73.32,18.3,18.32,19.7,24.4,...,0.0,11.8,12.7,14.7,19.4,3.9,-7,8.0,15,1
3,Unit,1,111.6,87.12,63.4,47.36,16.5,8.72,17.5,12.0,...,8.0,15.3,15.3,12.9,9.3,5.9,8,8.3,11,0
4,Skyler,1,122.0,122.02,77.3,82.60,19.8,26.02,18.0,12.8,...,25.2,11.3,4.5,14.7,8.1,3.9,11,7.9,2,0
5,Birdy,1,127.7,123.96,88.7,105.80,20.8,38.76,15.4,17.9,...,6.3,11.3,6.7,19.0,12.1,6.5,5,8.2,3,1
6,Fraus,1,93.0,81.42,59.7,63.20,20.0,8.82,17.7,23.2,...,14.8,15.0,12.1,0.2,0.0,5.3,14,8.7,5,0
7,Sierra,1,105.6,86.90,57.8,38.70,15.3,10.90,18.4,15.0,...,9.3,7.6,12.5,10.7,16.2,5.9,2,8.5,7,1
8,Elijah,1,128.5,68.84,89.8,104.52,18.5,6.84,15.7,13.9,...,9.0,11.9,1.5,16.6,1.8,6.9,4,8.1,6,0
9,Zander,1,128.3,127.98,80.3,70.22,19.0,22.58,20.9,18.4,...,9.1,11.3,13.9,13.5,12.3,8.3,1,8.6,8,1


In [45]:
major_stats = ['Score','Bench','QB','RB1','RB2','WR1','WR2','TE','FLEX','D','K']

Week by Week stats

In [46]:
total_weeks = raw_data['Week'].max()

for week in range(total_weeks, total_weeks + 1):
    week_data = raw_data[raw_data['Week'] == week]
    print(f"processing week {week} ...")
    for stat in major_stats:
        print(f'Processing {stat}...')
        data = {
            "line_x": [],
            "line_y": [],
            "Predicted": [],
            "Actual": []
        }

        # I have no idea why all this is necessary, I'm just following the documentaion
        players = (
            week_data.sort_values(by=[f'{stat} Actual'], ascending=True)['Name'].unique()
        )

        marker_color_actual = []

        for player in players:
            pred = week_data.loc[week_data['Name'] == player][f'{stat} Projected'].values[0]
            act = week_data.loc[week_data['Name'] == player][f'{stat} Actual'].values[0]
            data["Predicted"].append(pred)
            data["Actual"].append(act)
            data["line_x"].extend([pred, act, None])
            data["line_y"].extend([player,player,None])
            marker_color_actual.append('green' if act > pred else 'crimson')

        fig = go.Figure(data = [
            go.Scatter(
                x = data["line_x"],
                y = data["line_y"],
                mode = 'markers+lines',
                showlegend = False,
                marker=dict(
                    symbol='arrow',
                    color='black',
                    angleref='previous',
                    standoff=8,
                    size=16
                )
            ),
            go.Scatter(
                x = data['Predicted'],
                y = players,
                mode = 'markers+text',
                name = 'Predicted',
                marker = dict (color='silver',size=16),
                text = [str(x) for x in data['Predicted']],
                textposition = 'bottom center',
                showlegend = False,
            ),
            go.Scatter(
                x = data['Actual'],
                y = players,
                mode = 'markers+text',
                name = 'Actual',
                marker = dict (color=marker_color_actual, size=16),
                text = [str(x) for x in data['Actual']],
                textposition = 'top center',
                showlegend = False,
            )
        ])
        fig.update_layout(
            title = f'Week {week} Fantasy Football {stat} Scores: Projected vs Actual',
            height = 75 * len(players)
        )
        os.makedirs(f'Graphs/week_{week}', exist_ok=True)
        fig.write_image(f'Graphs/week_{week}/{stat}.png',height=75 * len(players))

processing week 1 ...
Processing Score...
Processing Bench...
Processing QB...
Processing RB1...
Processing RB2...
Processing WR1...
Processing WR2...
Processing TE...
Processing FLEX...
Processing D...
Processing K...
